In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from datasets import load_from_disk

import numpy as np
import json, copy, os, sys
from tqdm import tqdm
from IPython.display import display, HTML

import logging
logger = logging.getLogger('logger')
logging.basicConfig(level=logging.INFO)
logger.setLevel(logging.INFO)

sys.path.append(os.path.abspath(os.getcwd()))
from annotation_utilities import *

# this is the list of phenomena and which option they need to be annotated with:
phenomena = {
    'addition':'annotate_word',
    'ambiguous-translation-wrong-discourse-connective-since-causal':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-since-temporal':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-while-contrast':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-while-temporal':'diff_flexible',
    'ambiguous-translation-wrong-gender-female-anti':'diff_flexible',
    'ambiguous-translation-wrong-gender-female-pro':'diff_flexible',
    'ambiguous-translation-wrong-gender-male-anti':'diff_flexible',
    'ambiguous-translation-wrong-gender-male-pro':'diff_flexible',
    'ambiguous-translation-wrong-sense-frequent':'diff_flexible',
    'ambiguous-translation-wrong-sense-infrequent':'diff_flexible',
    'anaphoric_group_it-they:deletion':'annotate_word',
    'anaphoric_group_it-they:substitution':'annotate_word',
    'anaphoric_intra_non-subject_it:deletion':'annotate_word',
    'anaphoric_intra_non-subject_it:substitution':'annotate_word',
    'anaphoric_intra_subject_it:deletion':'annotate_word',
    'anaphoric_intra_subject_it:substitution':'annotate_word',
    'anaphoric_intra_they:deletion':'annotate_word',
    'anaphoric_intra_they:substitution':'annotate_word',
    'anaphoric_singular_they:deletion':'annotate_word',
    'anaphoric_singular_they:substitution':'annotate_word',
    'antonym-replacement':'REF_flexible',
    'commonsense-only-ref-ambiguous':'diff_flexible',
    'commonsense-src-and-ref-ambiguous':'diff_flexible',
    'copy-source':'whole_sentence',
    'coreference-based-on-commonsense':'mixed_flexible',
    'do-not-translate':'whole_sentence',
    'hallucination-date-time':'date',
    'hallucination-named-entity-level-1':'diff_flexible',
    'hallucination-named-entity-level-2':'REF_flexible',
    'hallucination-named-entity-level-3':'REF_flexible',
    'hallucination-number-level-1':'diff_flexible',
    'hallucination-number-level-2':'REF_flexible',
    'hallucination-number-level-3':'REF_flexible',
    'hallucination-real-data-vs-ref-word':'diff_flexible',
    'hallucination-real-data-vs-synonym':'diff_flexible',
    'hallucination-unit-conversion-amount-matches-ref':'units',
    'hallucination-unit-conversion-unit-matches-ref':'units',
    'hypernym-replacement':'REF_flexible',
    'hyponym-replacement':'REF_flexible',
    'lexical-overlap':'manual',
    'modal_verb:deletion':'add-omit',
    'modal_verb:substitution':'diff_flexible',
    'nonsense':'REF_flexible',
    'omission':'annotate_word',
    'ordering-mismatch':'swap',
    'overly-literal-vs-correct-idiom':'diff_flexible',
    'overly-literal-vs-explanation':'diff_flexible',
    'overly-literal-vs-ref-word':'diff_flexible',
    'overly-literal-vs-synonym':'diff_flexible',
    'pleonastic_it:deletion':'annotate_word',
    'pleonastic_it:substitution':'annotate_word',
    'punctuation:deletion_all':'add-omit',
    'punctuation:deletion_commas':'add-omit',
    'punctuation:deletion_quotes':'add-omit',
    'punctuation:statement-to-question':'add-omit',
    'real-world-knowledge-entailment':'diff_flexible',
    'real-world-knowledge-hypernym-vs-distractor':'diff_flexible',
    'real-world-knowledge-hypernym-vs-hyponym':'diff_flexible',
    'real-world-knowledge-synonym-vs-antonym':'diff_flexible',
    'similar-language-high':'whole_sentence',
    'similar-language-low':'whole_sentence',
    'untranslated-vs-ref-word':'diff_flexible',   # here add-omit can be used for getting character level replacements too
    'untranslated-vs-synonym':'diff_flexible',
    'xnli-addition-contradiction':'manual',
    'xnli-addition-neutral':'manual',
    'xnli-omission-contradiction':'manual',
    'xnli-omission-neutral':'manual'
}

folder = os.getcwd()
manual_annotations = os.path.join(folder, 'manual_annotations')
if not os.path.exists(manual_annotations):
    os.mkdir(manual_annotations)
    
phenomena_tobe_processed = input("enter the phenomena: ") 
if phenomena_tobe_processed == 'test':
    # load the subset.json
    dataset_path = os.path.join(manual_annotations, 'subset.json')
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s' %(dataset_path))
        exit()
    logger.info('Loading the test dataset...')
    with open(dataset_path, "r") as f:
        samples = json.load(f)
    logger.info('Test dataset loaded.')
elif phenomena_tobe_processed == 'dataset':
    dataset_path = os.path.join(folder, '../../dataset')
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s' %(dataset_path))
        exit()
    logger.info('Loading the dataset...')
    dataset = load_from_disk(dataset_path)
    logger.info('Dataset loaded.')
elif phenomena_tobe_processed in phenomena.keys():
    dataset_path = os.path.join(folder, '../../dataset')
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s' %(dataset_path))
        exit()
    logger.info('Loading the dataset...')
    dataset = load_from_disk(dataset_path)
    logger.info('Dataset loaded.')
    samples = dict()
    for idx, sample in enumerate(dataset['train']):
        if sample['phenomena'] in phenomena_tobe_processed:
            samples[idx] = sample        
else:
    logger.error("The phenomena should be one of these: {}".format(sys.argv[1], phenomena.keys()))
        
checkpoint = os.path.join(folder, 'manual_annotations/annotated_checkpoint_{}.txt'.format(phenomena_tobe_processed))
if os.path.exists(checkpoint):
    logger.info('Path {} already exists. Loading..'.format(checkpoint))
    annotations = read_json(checkpoint)
    annotations = {int(k):v for k,v in annotations.items()}
else:
    annotations = dict()

# calculate statistics about the annotations:
# for every mode, calculate no. of skipped, no. of unsure and ids, and no. of done.
stats_template = {
            'total':0,
            'success':0,
            'too_long':[],
            'no_change':[],
            'error':[],
            'other':[]  
        }

logger.info('Creating new stats.txt file')
stats = {}
for key in [phenomena_tobe_processed]:
    stats[key] = copy.deepcopy(stats_template)

res = input("Disable automatic accept (True to disable and anything else to not disable): ") 
automatic_accept = True
if res.lower() == 'true':
    automatic_accept = False
    print('Automatic accept disabled')
logger.info("READY")

def display_annotation(idx, sample):
    m1 = "<mark>"
    m2 = "</mark>"
    m_len = len(m1) + len(m2)
    n_spans = 0
    change = sample["annotation"]
    bad_new = sample["incorrect-translation"]
    for c in change:
        if c["in_bad"] != None:
            span = c["in_bad"]["character_span"]
            indices = (span[0]+n_spans*m_len, span[1]+n_spans*m_len)
            bad_new = bad_new[:indices[0]] + m1 + bad_new[indices[0]:indices[1]] + m2 + bad_new[indices[1]:]
            n_spans += 1
    html = '''
    <style>
    mark {{
        background-color: #4CAF50;
        color: black;
      }}
    </style>
    <body>
    <h3>{}</h3>
    <p>Source: {}</p>
    <p>Reference: {}</p>
    <p>Good translation: {}</p>
    <p>Incorrect: {}</p>
    <p>Phenomenon: {}</p>
    </body>
    '''.format(idx, sample['source'], sample['reference'], sample['good-translation'], bad_new, sample['phenomena'])
    display(HTML(html))
    
def display_sample(idx, sample):
    html = '''
    <style>
    mark {{
        background-color: #4CAF50;
        color: black;
      }}
    </style>
    <body>
    <h3>{}</h3>
    <p>Source: {}</p>
    <p>Reference: {}</p>
    <p>Good translation: {}</p>
    <p>Incorrect: {}</p>
    <p>Phenomenon: {}</p>
    </body>
    '''.format(idx, sample['source'], sample['reference'], sample['good-translation'], sample['incorrect-translation'], sample['phenomena'])
    display(HTML(html))
    
def display_annotation_old(idx, sample):
    print("\nID: ", idx)
    print("Source sentence: ", sample['source'])
    print("Reference: ", sample['reference'])
    print("Good Translation: ", sample['good-translation'])
    print("Incorrect Translation: ", sample['incorrect-translation'])
    print('Suggested annotation:')
    print(annotations[idx]['annotation'], '\n')
    
# the UI (?) part of the annotation in general (ask if they want to accept the annotation, call manual_annotation if no)
def manual_annotation_io(idx):
    sample = samples[idx]
    if idx in annotations:
        change = annotations[idx]['annotation']   # now it's normalized annotation.
        if automatic_accept and len(change) == 1 and len(change[0]["in_good"]['token_index']) == len(change[0]["in_bad"]['token_index']):
            return 0
    if phenomena[sample["phenomena"]] in ['?', 'mixed_flexible']:
        print("-----> For this sample we can compare the Incorrect translation with either Reference or Good translation.")
    elif phenomena[sample["phenomena"]] in ['REF_flexible']:
        print("-----> For this sample we compare the Incorrect translation with the Reference.")
    else:
        print("-----> For this sample we compare the Incorrect translation with the Good translation.\n")
    if idx in annotations:
        display_annotation(idx, sample)
        inp = input('To accept the suggested annotation click on enter. To skip this one enter skip. To exit enter exit and enter anything else to manually annotate:')
        if inp == "skip":
            annotations.pop(idx)
            return 1  # this means, we are skipping, so should delete this annotation and then continue with the next.
        if inp == "exit":
            # do not add the annotation if you stop at this point
            annotations.pop(idx)
            return -1
        res = manual_annotation(idx, inp)
        if res == -1:
            # do not add the annotation if you stop at this point
            annotations.pop(idx)
            return -1
        if res == 1:
            # skipping
            annotations.pop(idx)
            return 1
    else:
        display_sample(idx, sample)
        print("No automatic translations for this sample.")
        res = manual_annotation(idx)
        if res == -1:
            return -1

# the UI (?) part of the manual annotation
def manual_annotation(idx, inp="."):
    while inp != "":
        sample = samples[idx]
        inp = input("Enter the incorrect translation with the < and > to show the error spans (exit to stop, skip to skip): \n")
        bad = inp
        if bad == "exit":
            return -1
        if bad == "skip":
            return 1
        inp = input("Enter the correct/reference translation with the < and > to show the error spans (exit to stop, skip to skip): \n")
        good = inp
        if good == "exit":
            return -1
        if good == "skip":
            return 1
        
        change = calculate_change(good, bad, sample)
        tmp = copy.deepcopy(sample)
        tmp["annotation"] = change
        display_annotation(idx, tmp)
        inp = input("\n To accept it press enter or to annotate again enter any other string: ")
        if inp == "":
            sample['annotation'] = change
            sample['method'] = "manual annotation"
            annotations[idx] = sample
    return annotations[idx]

# given a manually annotated sample (where there are <> in incorrect and good/reference sentences)
# calculate the character spans in the original sentences and return the change in our annotation format
def calculate_change(good, bad, sample):  
    bad_id = 0
    span = False # False is when we are not inside a span, True is inside a span
    change = []
    for i, c in enumerate(bad):
        if c == "<":
            if span:
                logger.error("< not closed. Try again.\n")
                return manual_annotation(".", sample)
            else:
                start = bad_id
                start_annotate = i
                bad_id -= 1
                span = True
        elif c == ">":
            if not span:
                logger.error("No opening < Try again.\n")
                return manual_annotation(".", sample)
            else:
                change.append({"in_good":None, 
                    "in_bad":{'token_index':None, 
                    'character_span':(start,bad_id), 
                               'token':bad[start_annotate+1:i]}})
                bad_id -= 1
                span = False
        bad_id += 1
    good_id = 0
    span = False # False is when we are not inside a span, True is inside a span
    for i, c in enumerate(good):
        if c == "<":
            if span:
                logger.error("< not closed. Try again.\n")
                return manual_annotation(".", sample)
            else:
                start = good_id
                start_annotate = i
                good_id -= 1
                span = True
        elif c == ">":
            if not span:
                logger.error("No opening < Try again.\n")
                return manual_annotation(".", sample)
            else:
                change.append({"in_good":{'token_index':None, 
                    'character_span':(start,good_id), 
                               'token':good[start_annotate+1:i]}, 
                    "in_bad":None})
                good_id -= 1
                span = False
        good_id += 1
    return change

# process given sample, annotate or do manual annotation (only in the annotations.ipynb, in process_dataset.py only automatic annotation)
def process_sample(idx, sample, manual=False, detokenize=False):
    if phenomena[sample["phenomena"]] == 'mixed_flexible':
        good_og = ref_or_good(sample["reference"], sample["good-translation"], sample["incorrect-translation"])
    elif phenomena[sample["phenomena"]] == 'REF_flexible':
        good_og = sample["reference"]
    else:
        good_og = sample["good-translation"]
    bad_og = sample["incorrect-translation"]
    # if detokenize we just annotate the detokenized sentences, then map the character span back to the original sentence
    # in the standardize_annotation function in annotation_utilities.py
    if detokenize:
        try:
            good, good_map = detokenize_text(good_og, lang=sample["langpair"].split('-')[1])
            bad, bad_map = detokenize_text(bad_og, lang=sample["langpair"].split('-')[1])
            maps = (good_map, bad_map)
        except:
            good, bad = good_og, bad_og
            maps = None
    else:
        good, bad = good_og, bad_og
        maps = None # the standardize_annotation function will understand that it does not need to revert detokenization 
        # if maps parameter is None.
    originals = (good_og, bad_og)
    
    if phenomena[sample["phenomena"]] == 'add-omit':
        try:
            change = diff_char_level(good, bad)
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx))
            else:
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except:
            logger.warning('error in char level annotate, id {}'.format(idx))

    elif phenomena[sample["phenomena"]] == 'annotate_word':
        try:
            change = annotate_word(good, bad)
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            elif len(change) > 5:
                stats[sample["phenomena"]]["too_long"].append(idx)
                change = standardize_annotation(change, good, bad, maps, originals)
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except:
            logger.warning('error in word level annotate, id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] in ['diff_flexible', 'REF_flexible', 'mixed_flexible']:
        g, g_spans = tokenize(good)
        b, b_spans = tokenize(bad)

        # special treatment to japanese chinese and thailandish because they don't use spaces, so can't be split            
        if sample['langpair'][-2:] not in ['ja', 'zh', 'th']:      
            if len(g) == len(b):   # if there are multiple one word replacements
                change = diff(g, g_spans, b, b_spans, phenomena="replacement")
            if len(g) != len(b) or len(change) == 0:
                try:
                    change = diff_flexible(good, g, g_spans, bad, b, b_spans)
                    if len(change) == 0 and good != bad:
                        change = diff_char_level(good, bad) 
                except:
                    logger.warning('error in id {}'.format(idx))
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx,g,b,change))
            elif len(change) != 0 and ((change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 50) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 50)):
                logger.warning('check this - too long: %s' %idx)
            else:
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample  
        else:
            try:
                change = diff_char_level(good, bad) 
                if len(change) == 0 and good != bad:
                    logger.warning('No change in id {}'.format(idx,g,b,change))
                elif len(change) != 0 and ((change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 30) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 30)):
                    logger.warning('check this - too long: %s' %idx)
                else:
                    change = standardize_annotation(change, good, bad, maps, originals)
                sample['annotation'] = change
                sample['method'] = phenomena[sample["phenomena"]]
                annotations[idx] = sample
            except: 
                logger.warning('error in id {}'.format(idx))

    elif phenomena[sample["phenomena"]] == 'units':
        try:
            g, b, change = annotate_units(good,bad)
            if len(change) == 0 and g != b:
                logger.warning('No change in id {}, \ng: {}, \nb: {},\nr: {}'.format(idx, g, b))
            elif len(change) > 1:
                logger.warning('Multiple changes in {} id {}'.format(sample["phenomena"], idx))
            else:
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample  
        except: 
            logger.warning('error in id {}'.format(idx))

    elif phenomena[sample["phenomena"]] == 'swap':
        try:
            change = annotate_swap_word_lvl(good,bad)
            if len(change) < 2 and good != bad:
                logger.warning('No change in id {}, \ng: {}, \nb: {}'.format(idx, good, bad))
            elif change[0]['in_good'] != None and change[1]['in_good'] != None and change[0]['in_good'] == change[1]['in_good']:
                logger.warning('check this: %s - swapped words are the same!' %idx)
            elif (change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 50) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 50):
                logger.warning('check this: %s' %idx)
            else:
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except: 
            logger.warning('error in id {}'.format(idx))

    elif phenomena[sample["phenomena"]] == 'date':
        try:
            change = diff_dates(good,bad)
            change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except: 
            logger.warning('error in id {}'.format(idx))
    elif phenomena[sample['phenomena']] == 'whole_sentence':
        change = whole_sentence(good, bad)
        change = standardize_annotation(change, good, bad, maps, originals)
        sample['annotation'] = change
        sample['method'] = phenomena[sample["phenomena"]]
        annotations[idx] = sample
    if manual:
        # if the function returns 0 it automatically accepted, if 1 skipping and if -1 exit
        res = manual_annotation_io(idx)
        if res == 1:  # SKIPPING
            return 1 
        # if exit, first save a new annotations file to save progress and then exit
        if res == -1:
            save_json(annotations, checkpoint)
            return -1
    return 1  # 1 for success
        
def process_phenomena(samples, manual=False, detokenize=False):
    for idx,sample in tqdm(samples.items()):
        if idx not in annotations.keys() and int(idx) not in annotations.keys():            
            # check if it was annotated before
            res = check_seen_before(sample, annotations)
            if res != None:
                sample['annotation'] = res[0]
                sample['method'] = res[1]
                annotations[int(idx)] = sample
            else:
                try:
                    res = process_sample(idx, sample, manual, detokenize)
                except:
                    logger.error(idx)
                if res == -1:
                    return -1
    # save all annotations after finished
    save_json(annotations, checkpoint)


INFO:logger:Loading the dataset...
INFO:logger:Dataset loaded.
INFO:logger:Creating new stats.txt file
INFO:logger:READY


In [2]:
sys.path

['/mnt/c/Users/user/Desktop/work/ACES_private/challenge_set_annotation',
 '/home/arnisa/anaconda3/envs/annotate_env/lib/python38.zip',
 '/home/arnisa/anaconda3/envs/annotate_env/lib/python3.8',
 '/home/arnisa/anaconda3/envs/annotate_env/lib/python3.8/lib-dynload',
 '',
 '/home/arnisa/anaconda3/envs/annotate_env/lib/python3.8/site-packages',
 '/mnt/c/Users/user/Desktop/work/ACES_private/challenge_set_annotation/ACES_private/span_predictions']

In [1]:
import os, sys, argparse, logging, json, glob
from tqdm import tqdm
from datasets import load_from_disk
import pandas as pd
logger = logging.getLogger('logger')
logging.basicConfig(level=logging.INFO)

sys.path.append(os.path.abspath("../span_predictions"))
from format_utilities import *
sys.path.append(os.path.abspath("../challenge_set_annotation"))
from annotation_utilities import *
sys.path.append(os.path.abspath("../aces"))
from utils import read_file

/home/arnisa/anaconda3/envs/annotate_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
def annotation_to_ACES_sample(idx, sample, manual=False):  
    sample_out = sample.copy()
    sample_out["incorrect-translation-annotated"] = change_to_tsv_annotation(sample, m1="<v>", m2="</v>")
    if manual:
        sample_out["annotation-method"] = True
    else:
        sample_out["annotation-method"] = sample["method"]
    del sample_out["method"]
    if "omission" in sample_out:
        del sample_out["omission"]
    del sample_out["annotation"]
    return sample_out

In [41]:
annotations = read_json("/mnt/c/Users/user/Desktop/work/ACES_private/challenge_set_annotation/annotated.txt")
annotations = {int(k):v for k,v in annotations.items()}
samples = {}
for (idx,sample) in tqdm(annotations.items()):
    samples[idx] = annotation_to_ACES_sample(idx, sample)

100%|██████████| 34470/34470 [00:00<00:00, 237313.28it/s]


In [43]:
samples[6]

{'source': '"Fire captain Scott said, ""It was a hot day in the Santa Clara with temperatures in the 90s."',
 'good-translation': '"El capitán de bomberos Scott dijo: ""Fue un día caliente en Santa Clara con temperaturas en los años 90."',
 'incorrect-translation': '"El capitán de bomberos Scott Kouns dijo: ""Fue un día caliente en Santa Clara con temperaturas en los años 90."',
 'reference': 'Scott , capitán del cuartel de bomberos, comentó: «Era un día de calor en Santa Clara, con temperaturas de 90 grados.',
 'phenomena': 'addition',
 'langpair': 'en-es',
 'incorrect-translation-annotated': '"El capitán de bomberos Scott <v>Kouns</v> dijo: ""Fue un día caliente en Santa Clara con temperaturas en los años 90."',
 'annotation-method': 'annotate_word'}

In [73]:
df = pd.DataFrame.from_dict(samples, orient='index', columns=['source', 'good-translation', 'incorrect-translation', 'reference',  'phenomena', 'langpair', 'incorrect-translation-annotated', 'annotation-method'])
# Optional: Rename the index column
df.index.name = 'ID'

# Display the DataFrame
print(df)

                                                  source  \
ID                                                         
0      Proper nutritional practices alone cannot gene...   
1       has geographic variations, where the age limi...   
2      The U.N. also hopes to finalize a fund to help...   
3      Several million vials of vaccine have also bee...   
4      Pre-operative people should not expect to pass...   
...                                                  ...   
29951    彼はノッティンガム、セルストンのブルックの娘、メアリーと結婚し、長男のリチャードに引き継いだ。   
32324             움직임은 3x4에서 공통 섹션이 있긴 하지만 장기간에 걸쳐 있습니다.   
32328  João Martins Pena와 Francisca de Paula Julieta ...   
32337  하계 대학 리그의 일부인 Nashua Silver Knights는 이 도시의 현재 ...   
32339  그는 체코의 국가 대표 선수 Jaroslav Hlinka의 사촌이자 한 때 선수였던...   

                                        good-translation  \
ID                                                         
0      Las prácticas nutricionales adecuadas por sí s...   
1      tiene variaciones geográficas, d

In [26]:
def load_dataset(dataset_path):
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s. See ACES_private/challenge_set_annotation/download_dataset.py to download it from HuggingFace' %(dataset_path))
        exit()
    logger.info('Loading the dataset...')
    dataset = load_from_disk(dataset_path)
    logger.info('Dataset loaded.')
    return dataset['train']
dataset = load_dataset("/mnt/c/Users/user/Desktop/work/dataset")
print('Full dataset size: ', len(dataset))

INFO:logger:Loading the dataset...
INFO:logger:Dataset loaded.


Full dataset size:  36476


In [65]:
content

,ID,TYPE,LANG,A,B,REANNOTATION_REASON
0,3038,word swap,en,"The movement is in extended time, although the...",The movement is in extended time although ther...,Unannotated
1,3050,word swap,en,João Martins Pena and Francisca de Paula Julie...,"<Was born> Martins Pena in Rio de Janeiro, <Jo...",Unannotated
2,3060,word swap,en,"The Nashua Silver Knights, part of a summer co...","The Nashua Silver Knights, part of a <current>...",Unannotated
3,6329,word swap,en,"He married Mary, the daughter of Brooke of Sel...","He married Mary, the daughter of Timothy Pusey...",Unannotated
4,6348,word swap,en,"The movement is in extended time, although the...",The movement is in extended time although ther...,Unannotated
5,6356,word swap,en,A CD with themes from 14 of his films was rele...,A CD with themes from fourteen of his films wa...,Unannotated
6,6375,word swap,en,"The Nashua Silver Knights, part of a summer co...","The Nashua Silver Knights, part of a <current>...",Unannotated
7,10259,word swap,en,"The movement is in extended time, although the...",The movement is in extended time although ther...,Unannotated
8,10265,word swap,en,A CD with themes from 14 of his films was rele...,A CD with themes from fourteen of his films wa...,Unannotated
9,29944,word swap,en,"The Nashua Silver Knights, part of a summer co...","The Nashua Silver Knights, part of a <current>...",Unannotated


In [47]:
df.head()

,source,good-translation,incorrect-translation,reference,phenomena,langpair,incorrect-translation-annotated,annotation-method
ID,,,,,,,,
0,Proper nutritional practices alone cannot gene...,Las prácticas nutricionales adecuadas por sí s...,Las prácticas nutricionales adecuadas por sí s...,No es posible que las prácticas nutricionales ...,addition,en-es,Las prácticas nutricionales adecuadas por sí s...,annotate_word
1,"has geographic variations, where the age limi...","tiene variaciones geográficas, donde el límite...","La definición tiene variaciones geográficas, d...","varía geográficamente, donde el límite de eda...",addition,en-es,<v>La definición</v> tiene variaciones geográf...,annotate_word
2,The U.N. also hopes to finalize a fund to help...,La ONU también espera finalizar un fondo para ...,La ONU también espera finalizar un fondo para ...,La ONU tiene la intención de completar un fond...,addition,en-es,La ONU también espera finalizar un fondo para ...,annotate_word
3,Several million vials of vaccine have also bee...,Varios millones de frascos de vacuna también h...,Varios millones de frascos de vacuna contra la...,Las autoridades también han garantizado varios...,addition,en-es,Varios millones de frascos de vacuna <v>contra...,annotate_word
4,Pre-operative people should not expect to pass...,Las personas preoperativas no deben esperar pa...,Las personas transgénero preoperativas no debe...,Las personas que aún no se han operado no debe...,addition,en-es,Las personas <v>transgénero</v> preoperativas ...,annotate_word


In [60]:
content

,level_0,index,ID,TYPE,LANG,A,B,REANNOTATION_REASON
0,0,0,19552,hallucination,de,"Zuvor bemerkte der CEO von Ring, Jamie Siminof...","Zuvor bemerkte der CEO von Ring, Jamie Siminof...",Unannotated
1,1,1,25218,hallucination,de,Aber nach dem Verlust des Schalters seines Kap...,Aber nach dem Verlust des Schalters seines Kap...,Unannotated
2,2,2,25253,hallucination,de,Virtuelle Steckdosen sind in die Software inte...,Virtuelle Steckdosen sind in die Software inte...,Unannotated


In [72]:
file = "/mnt/c/Users/user/Desktop/work/manual_annotations/Annotation_SecondDelivery/annotated_EN_for_reannotation.tsv"


In [68]:
samples[32337]

{'source': '하계 대학 리그의 일부인 Nashua Silver Knights는 이 도시의 현재 팀이다.',
 'good-translation': "The Silver Knights of Nashau (part of the Summer University Alliance) are now the city's team.",
 'incorrect-translation': 'The Nashua Silver Knights, part of a current summer league, is the collegiate team of the city.',
 'reference': "The Nashua Silver Knights, part of a summer collegiate league, is today's team in the city.",
 'phenomena': 'lexical-overlap',
 'langpair': 'ko-en',
 'incorrect-translation-annotated': 'The Nashua Silver Knights, part of a <v>current</v> summer league, is the <v>collegiate</v> team of the city.'}

In [6]:
# run this cell to start from the beginning - not from a checkpoint (you will lose the prev checkpoint tho)
annotations = dict()

In [7]:
# THE ACTUAL PART
logger.setLevel(logging.INFO)
stats = {}
for key in [phenomena_tobe_processed]:
    stats[key] = copy.deepcopy(stats_template)
process_phenomena(samples, manual=False, detokenize=True)

100%|██████████| 999/999 [00:10<00:00, 95.50it/s] 


In [82]:
annotations[6]

{'source': '"Fire captain Scott said, ""It was a hot day in the Santa Clara with temperatures in the 90s."',
 'good-translation': '"El capitán de bomberos Scott dijo: ""Fue un día caliente en Santa Clara con temperaturas en los años 90."',
 'incorrect-translation': '"El capitán de bomberos Scott Kouns dijo: ""Fue un día caliente en Santa Clara con temperaturas en los años 90."',
 'reference': 'Scott , capitán del cuartel de bomberos, comentó: «Era un día de calor en Santa Clara, con temperaturas de 90 grados.',
 'phenomena': 'addition',
 'langpair': 'en-es',
 'annotation': ([{'in_bad': {'token_index': [6],
     'character_span': (30, 35),
     'token': 'Kouns'},
    'in_good': None}],
  False),
 'method': 'annotate_word'}

In [83]:
with open(checkpoint, "w+") as f:
    json.dump(str(annotations), f, indent=2, ensure_ascii=False)  # encode dict into JSON

In [86]:
with open(checkpoint, "r") as f:
    annotations = json.load(dict(f))

ValueError: dictionary update sequence element #0 has length 782572; 2 is required

In [87]:
import json

# Your data to be serialized
data = {
    "name": "John",
    "description": "Contains special characters like é, ç, and ∆."
}



# Serialize the data to a JSON file with the custom encoder


In [88]:
import json

# Define a custom JSON decoder to handle special characters


# Read the data from the JSON file into a dictionary


# Now, 'loaded_data' contains the dictionary from the JSON file, including special characters
print(loaded_data)

{'name': 'John', 'description': 'Contains special characters like é, ç, and ∆.'}


In [ ]:
# After this cell there are extra stuff - no need to run

In [109]:
# To run normalization over the already annotated samples in coreference-based-on-commonsense
# No need to do that for manual annotation!
for idx in annotations:
    sample = annotations[idx]
    good, _, _ = ref_or_good(sample["reference"], sample["good-translation"], sample["incorrect-translation"])
    bad = sample['incorrect-translation']
    change = sample['annotation']
    try:
        sample['annotation'] = standardize_annotation(change, good, bad)
    except:
        print(idx)

In [112]:
with open(checkpoint, "w+") as f:
    json.dump(annotations, f, indent=2, ensure_ascii=False)  # encode dict into JSON

In [2]:
# for checking if all langpairs are correct
langpairs = set()
for sample in dataset["train"]:
    langpairs.add(sample["langpair"])

In [6]:
langpairs_keys = list(langpairs)

In [ ]:
# to count the number of samples in any phenomena
# No need to do that for manual annotation!
basic_lang = ['en', 'de', 'fr', 'mr', 'tr']
for p in ['coreference-based-on-commonsense', 'lexical-overlap',
         'xnli-addition-contradiction', 'xnli-addition-neutral',
         'xnli-omission-contradiction', 'xnli-omission-neutral']:
    print("**********************", p)
    l_found = dict(zip(langpairs_keys, np.zeros(len(langpairs))))
    for sample in dataset["train"]:
        src, tgt = sample["langpair"].split('-')
        if sample['phenomena'] == p and l_found[sample["langpair"]] == 0 and (src not in basic_lang or tgt not in basic_lang):
            print(sample["langpair"], ": \n\t", sample['source'], "\n\t", sample['good-translation'])
            l_found[sample["langpair"]] = 1